## 当前状态: 代码已移到centos中, 用自带的服务器运行无报错

## wsgi
- wsgi全称: web server gateway interface
- 作用是充当Nginx, Apache等服务器和开发者写的application的桥梁, 负责调用application, 同时把request作为参数传给application

## app server VS web server(有误, 待验证)
- app server: Gunicorn
- web server: Apache, Nginx

## 为什么生产环境不用Django自带的web server?(manage.py runserver VS gunicorn)
1. [官方文档](https://docs.djangoproject.com/en/2.0/ref/django-admin/)指出:
> DO NOT USE THIS SERVER IN A PRODUCTION SETTING. It has not gone through security audits or performance tests. (And that’s how it’s gonna stay. We’re in the business of making Web frameworks, not Web servers, so improving this server to be able to handle a production environment is outside the scope of Django.)
2. Django自带的server是单线程的, 这意味着没有并发, 而且任何一个代码的hangup都会导致server的hangup.
3. manage.py runserver和gunicorn的作用一样, 只是manage.py runserver面向development, gunicorn面向production
4. gunicorn可以同时平行地spawn多个worker processes到多个cpu.
5. gunicorn有更好的logging
6. gunicorn会主动处理安全问题

----

## nginx和gunicorn有什么区别(niginx VS gunicorn)
1. 总体的流程是:
    * nginx接受client的请求
    * 如果是请求静态文件, nginx直接返回静态文件, 不需要经过django的app
    * 如果是动态请求, nginx把请求交给gunicorn, gunicorn再调用django app, 把结果返回给nginx, nginx再返回给client
    
2. 所以gunicorn是nginx和django app的连接部分, 所以叫做wsgi(web server gateway interface)
3. 实际上, 只用gunicorn, 不用nginx也是可以的


## 开始部署

1. ```pip3 install gunicorn```
2. ```gunicorn myproect.wsgi:application -b 0.0.0.0:8000```(注意: 要先开启8000端口, 具体开启方法看"Linux命令"中的笔记)
    * 可以成功访问, 但发现页面css样式没有加载
        + 静态文件一般的做法是由nginx加载
        + 在用```manage.py runserver```的时候没有这个问题, 是因为Dajngo自带了静态文件处理器
        + [这里](https://zhu327.github.io/2015/09/29/gunicorn%E8%BF%90%E8%A1%8Cdjango%E6%97%B6%E9%9D%99%E6%80%81%E6%96%87%E4%BB%B6%E5%A4%84%E7%90%86/)提供了不用nginx的情况下, 处理静态文件的做法
    * myproject指的是项目的名称, 其他都是固定的用法
3. 对于2的做法, 可以用```nohup gunicorn myproject.wsgi:application -b 0.0.0.0:8000 &```指令替代, 这样使server在后台运行, 注意, 该命令执行完后会返回一个进程号, 前台不会显示程序结果, 结果会输出到nohup.out文件中.
4. ```jobs -l```, 查看后台运行的进程
5. ```kill 进程号(PID)```: 杀死该进程
6. 处理静态文件
    * 在setting.py中设置静态文件目录